In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MaxAbsScaler
from pyGRNN import GRNN
from sklearn.cluster import KMeans
from lazypredict.Supervised import LazyRegressor

In [2]:
from sklearn.metrics import mean_absolute_error
from numpy import sqrt
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_percentage_error, max_error, median_absolute_error, mean_absolute_error

def print_metrics(y_true, y_pred):
    mse = mean_squared_error(y_true, y_pred)
    print(f'Mean Squared Error (MSE):              {mse:.10f}')
    
    rmse = sqrt(mse)
    print(f'Root Mean Squared Error (RMSE):        {rmse:.10f}')
    
    mae = mean_absolute_error(y_true, y_pred)
    print(f'Mean Absolute Error (MAE):             {mae:.10f}')
    
    r2 = r2_score(y_true, y_pred)
    print(f'R-squared (R²):                        {r2:.10f}')
    
    mape = mean_absolute_percentage_error(y_true, y_pred)
    print(f'Mean Absolute Percentage Error (MAPE): {mape:.10f}')    
    
    me = max_error(y_true, y_pred)
    print(f'Max Error (ME):                        {me:.10f}')    
    
    med_ae = median_absolute_error(y_true, y_pred)
    print(f'Median Absolute Error (MedAE):         {med_ae:.10f}') 

In [3]:
df = pd.read_csv('data/train.csv')
df.head(10)

,max_discharge_sum_r1,max_discharge_sum_r2_1,max_discharge_sum_r2_2,max_discharge_sum_r3,flood_sum_r1,flood_sum_r2_1,flood_sum_r2_2,flood_sum_r3,HW_inund_uncertainty,bridge_scour_uncertainty,travel_demand_uncertainty,restoration_capacity_uncertainty,dir_costs,lost_trips_costs_s_r,traffic_costs_s_r
0,585.67,1626.36,1917.92,638.86,8411.14,31506.11,22090.55,15338.74,1.03,1.14,0.76,6,33407655.48,63906033.49,-13876077.42
1,577.56,1643.06,1887.66,821.35,8428.71,31464.77,21934.20,17058.63,1.03,0.91,1.14,5,26222060.06,55359791.46,9384497.19
2,772.58,1672.02,1803.84,494.53,9338.14,31334.39,21498.21,13903.41,1.08,0.95,1.06,3,25265321.95,59332912.51,5139630.61
3,522.08,1653.12,3326.49,460.21,8055.62,40539.30,32937.77,13073.49,1.05,1.26,0.98,3,54860001.97,94212106.15,2127730.92
4,614.14,1644.18,2485.55,595.06,8529.56,34186.48,28422.80,14950.68,0.79,0.70,1.23,4,39212152.98,76321454.19,21132083.35
5,813.29,1669.06,1877.53,680.46,9701.22,31193.17,22297.85,15652.70,0.78,1.38,0.88,6,32258229.36,76162964.60,-6506723.41
6,663.21,1666.04,2230.32,671.19,8806.26,32504.62,26343.62,15618.30,0.88,0.74,0.64,6,35019441.00,32830011.84,-17554044.81
7,501.75,1726.18,2023.48,993.98,8031.13,32844.96,22531.50,18555.23,0.64,0.35,1.19,5,28815518.56,71389907.84,17423834.01
8,581.13,1669.38,1976.99,1105.23,8297.44,32213.13,22180.60,19413.06,0.97,0.52,0.94,6,31828474.78,56818034.31,-2533088.91
9,622.46,1636.29,2176.80,457.47,8532.34,32557.58,24402.16,13253.84,0.86,1.26,1.16,4,45656687.89,61532735.19,14043780.74


In [4]:
y1_name, y2_name, y3_name = "dir_costs","traffic_costs_s_r","lost_trips_costs_s_r" 
y1, y2, y3 = df[y1_name], df[y2_name], df[y3_name]

In [5]:
X = df.drop(columns=[y1_name, y2_name, y3_name])
scaler = MaxAbsScaler()
scaler.fit(X)
X_scaled = scaler.transform(X)

X_scaled_df = pd.DataFrame(X_scaled, columns=X.columns)

In [ ]:
from sklearn.model_selection import GridSearchCV, KFold
from pyGRNN import GRNN
import numpy as np

# Define the GRNN model
grnn1 = GRNN()

# Define the parameter grid for sigma values
param_grid = {'sigma': np.logspace(-2, 1, 10)}  # Adjust range and granularity as needed

# Define cross-validation strategy
kf = KFold(n_splits=5, shuffle=True, random_state=42)

# Define scoring metric (e.g., mean squared error)
scoring = 'neg_mean_squared_error'

# Perform grid search with cross-validation
grid_search = GridSearchCV(estimator=grnn1, param_grid=param_grid, scoring=scoring, cv=kf)
grid_search.fit(X_scaled_df, y1)

# Print the best sigma value and corresponding mean cross-validated score
print("Best Sigma:", grid_search.best_params_['sigma'])
print("Best Score (Negative MSE):", -grid_search.best_score_)

Executing warm start...
Warm start concluded. The optimum isotropic sigma is [0.07676997]
Gradient search concluded. The optimum sigma is [0.05765124 0.0120242  0.00591582 0.58679079 0.05581141 0.01343684
 0.04878113 0.24560255 0.44700383 0.69848175 0.51186836 0.35796405]
Executing warm start...
Warm start concluded. The optimum isotropic sigma is [0.07787263]
Gradient search concluded. The optimum sigma is [1.70986544e+01 1.97057467e-02 1.89763591e+01 1.02271079e+01
 5.88547391e-02 1.14514359e-02 1.63482222e-02 1.09784170e-01
 7.20501549e+00 3.04037023e-01 1.54241057e+01 1.90197041e-01]
Executing warm start...
Warm start concluded. The optimum isotropic sigma is [0.07971944]
Gradient search concluded. The optimum sigma is [nan nan nan nan nan nan nan nan nan nan nan nan]
Executing warm start...
Warm start concluded. The optimum isotropic sigma is [0.07348695]
Gradient search concluded. The optimum sigma is [0.04050668 0.01008986 0.00711998 0.9227825  6.70788552 0.63696555
 0.01297986 